# An Explain Report

This is a POC for an explain report: given a number at time T1 and time T2, introspect the clocks involved to produce a simple linear breakdown
of what changed on various timelines to account for the change in the number between T1 and T2.

The report is an ordered sequence of clock changes: the sum of changes should equal to the total change. The order of application is important: change the
ordering, and the resultant values may change. It's not a Jacobian-type report with T1 deltas and no ordering.

The real reason for this workbook is to start working on the node class optimizations and metadata management.

## Footnotes

Footnotes are one of the simplest types of metadata:
* Any computation (even a simple read of a constant value) may declare it has footnotes
* Every computation gets the footnote of all its inputs

Footnoting is based on a number of observations about large software systems (say, 1M LOC or more):
* Any complex report (i.e. computation output) will generally be somewhat wrong, misleading, or out of date.
* If you want 100% correctness and truth, your report will raise an exception every time you run it.
* If you just want it to run, you'll get a result, but some log file on some machine in some compute farm will have a message explaining why your result is wrong.
* And you won't see that message.
* And that message may have leaked information that bad actors can read.
* And you won't trust this system, so your own developers will copy the underlying data and write their own version of the report with all the same issues.
* Now you don't have a problem, but your company now has two problems.

Footnotes summary:
* Are the documentation of problems from the producer's point of view
* Make no claim about usability of results, etc, for a specific consumer
* May be programmatically removed/condensed/replaced at controlled code points


In [1]:
import mand.core

from mand.core import Entity, node, Context
from mand.core import ObjectDb, Timestamp, Context, getNode, _tr
from mand.core import ProfileMonitor
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import find
from mand.demos.trading import makeWorld, bookSomeTrades

db = ObjectDb()

from mand.lib.dbsetup import setUpDb
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x10bcb7350>: 231, mem=True, ro=False: entities=9, map=2


In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
#p3 = pAll.children()[2]
p4 = pAll.children()[3]

#b1 = p1.children()[0]
b2 = p2.children()[0]
#b3 = p3.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


# Book some trades

This time, we throw a few thousand in...

In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

# An Explain Report

This is a mix of abstractions at the Core, DBA, and BA, and User levels. But, it does the job for now...

In [4]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        # XXX - modifying tweaks in place is a bit evil
        # This is only safe because I know Context() effectively copies, so this works
        # for now.
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            txt = '\n'.join( [ '1. %s' % f for f in footnotes])
            displayMarkdown(txt)
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 5236.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [5]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

# Footnotes

Note the report calculation has run, but attached appropriate caveats to the output:

In [6]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times: /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Report:data|1|83,509,262|44|83,509,262|GetValue
|Report:data|1|83,509,217|1,833|83,509,217|GetValue/Calc
|TradingContainer:NPV|11|83,291,325|481|7,571,938|GetValue
|TradingContainer:NPV|11|83,290,843|5,702|7,571,894|GetValue/Calc
|Portfolio:items|121|82,674,153|4,012|683,257|GetValue
|Portfolio:items|121|82,673,807|102,814|683,254|GetValue/Calc
|Workbook:items|1,104|78,274,493|36,841|70,900|GetValue
|Workbook:items|1,100|78,237,652|34,440,106|71,125|GetValue/Calc
|Root:Clocks|2|52,703,429|200,712|26,351,714|Context
|TradeOpenEvent:ticket|22,286|24,145,190|500,702|1,083|GetValue
|TradingTicket|1,014|23,644,487|23,644,487|23,318|Db.Get
|TradeOpenEvent|1,014|16,559,197|16,559,197|16,330|Db.Get
|Root:Amend MarketData|1|6,657,507|546|6,657,507|Context
|Root:Start breaks|1|4,774,165|281|4,774,165|Context
|Portfolio:children|242|4,273,462|5,290|17,658|GetValue
|Portfolio:children|121|4,268,171|1,772,429|35,274|GetValue/Calc
|Root:Amend Portfolio|1|3,838,136|153|3,838,136|Context
|Root:Start|1|3,115,121|84|3,115,121|Context
|Root:Amend Trading|1|3,001,571|161|3,001,571|Context
|Root:Activity Portfolio|1|2,854,208|118|2,854,208|Context
|Root:Activity MarketData|1|2,528,877|126|2,528,877|Context
|PortfolioUpdateEvent:children|121|2,110,382|24,403|17,441|GetValue
|Root:End|1|2,026,770|120|2,026,770|Context
|Root:Activity Trading|1|1,993,953|113|1,993,953|Context
|TradingBook|102|1,979,119|1,979,119|19,403|Db.Get
|Equity:NPV|15|601,120|641|40,074|GetValue
|Equity:NPV|15|600,479|1,476|40,031|GetValue/Calc
|MarketInterface:spot|15|569,103|2,099|37,940|GetValue
|MarketInterface:spot|15|567,003|1,365|37,800|GetValue/Calc
|ExternalRefData:state|15|529,732|410|35,315|GetValue
|ExternalRefData:state|15|529,322|899|35,288|GetValue/Calc
|RefData:state|15|528,422|451|35,228|GetValue
|RefData:state|15|527,971|197,926|35,198|GetValue/Calc
|_WorkItemEvent:book1|22,286|469,224|469,224|21|GetValue
|_WorkItemEvent:book2|22,286|463,758|428,515|20|GetValue
|TradeOpenEvent:quantity|22,286|385,584|385,584|17|GetValue
|Clock:cutoffs|2,480|384,737|30,249|155|GetValue
|Clock:cutoffs|20|354,754|2,039|17,737|GetValue/Calc
|Clock:parent|20|352,314|706|17,615|GetValue
|Clock:parent|20|351,607|282,126|17,580|GetValue/Calc
|TradeOpenEvent:premium|11,143|332,222|316,431|29|GetValue
|_WorkItemEvent:item|11,143|325,173|295,243|29|GetValue
|PortfolioUpdateEvent|12|297,612|297,612|24,801|Db.Get
|TradeOpenEvent:action|11,143|293,666|293,666|26|GetValue
|TradeOpenEvent:unitPrice|11,143|246,295|246,295|22|GetValue
|Event:amends|11,340|227,774|227,774|20|GetValue
|TradingBook:clock|2,200|178,750|93,735|81|GetValue
|RefDataUpdateEvent|9|148,153|148,153|16,461|Db.Get
|TradingPortfolio|10|142,103|142,103|14,210|Db.Get
|TradingBook:clock|1,100|85,015|69,738|77|GetValue/Calc
|Clock|5|81,927|81,927|16,385|Db.Get
|MarketInterface:source|15|35,905|387|2,393|GetValue
|MarketInterface:source|15|35,518|1,296|2,367|GetValue/Calc
|ClockEvent:parent|8|33,749|394|4,218|GetValue
|MarketDataSource|2|33,643|33,643|16,821|Db.Get
|ClockEvent|2|32,890|32,890|16,445|Db.Get
|Equity|2|29,930|29,930|14,965|Db.Get
|Equity:refdata|15|29,899|453|1,993|GetValue
|Equity:refdata|15|29,445|2,078|1,963|GetValue/Calc
|MarketInterface|2|26,805|26,805|13,402|Db.Get
|Portfolio:clock|242|26,690|7,549|110|GetValue
|MarketDataSource:clock|30|21,882|1,142|729|GetValue
|MarketDataSource:clock|15|20,739|688|1,382|GetValue/Calc
|Portfolio:books|231|20,720|5,311|89|GetValue
|Portfolio:clock|121|19,140|5,895|158|GetValue/Calc
|Portfolio:books|121|16,561|14,058|136|GetValue/Calc
|ForwardCashflow|1|15,791|15,791|15,791|Db.Get
|RootClock|1|13,508|13,508|13,508|Db.Get
|ForwardCashflow:NPV|11|9,866|9,577|896|GetValue
|RefDataUpdateEvent:data|58|7,233|7,233|124|GetValue
|Entity:clock|40|2,127|1,299|53|GetValue
|Entity:clock|20|827|827|41|GetValue/Calc
|RootClock:cutoffs|52|648|648|12|GetValue
|MarketInterface:sourceName|15|577|424|38|GetValue
|Equity:assetName|15|562|562|37|GetValue
|ForwardCashflow:NPV|11|288|288|26|GetValue/Calc
|MarketInterface:sourceName|15|153|153|10|GetValue/Calc
|Report:valuable|1|87|87|87|GetValue
|Report:ts1|1|24|24|24|GetValue
|Report:ts2|1|22|22|22|GetValue